In [1]:
# My repo with data
!git clone https://github.com/Tixonmavrin/covid-19-solution
!unzip covid-19-solution/data/other/timeseries.csv.zip

Cloning into 'covid-19-solution'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 173 (delta 35), reused 0 (delta 0), pack-reused 97
Receiving objects: 100% (173/173), 5.28 MiB | 18.21 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Archive:  covid-19-solution/data/other/timeseries.csv.zip
  inflating: timeseries.csv          
  inflating: __MACOSX/._timeseries.csv  


In [0]:
import pandas as pd
import numpy as np

In [0]:
#data = pd.read_csv('covid-19-solution/data/data_clean/_nonoice_data_confirmed.csv')
#data_deaths = pd.read_csv('covid-19-solution/data/data_clean/_nonoice_data_deaths.csv')
#data_rus = pd.read_csv('covid-19-solution/data/data_clean/_nonoice_data_confirmed_rus.csv')
#data_deaths_rus = pd.read_csv('covid-19-solution/data/data_clean/_nonoice_data_deaths_rus.csv')

data = pd.read_csv('covid-19-solution/data/data_start/_data_confirmed.csv')
data_deaths = pd.read_csv('covid-19-solution/data/data_start/_data_deaths.csv')
data_rus = pd.read_csv('covid-19-solution/data/data_start/_data_confirmed_rus.csv')
data_deaths_rus = pd.read_csv('covid-19-solution/data/data_start/_data_deaths_rus.csv')

In [0]:
data = pd.read_csv('covid-19-solution/data/data_start/_data_confirmed_all.csv')
del data['Unnamed: 0']
data['Province/State'] = data['Province/State'].fillna('nan')

In [5]:
data

,Country/Region,Province/State,01/22/20,01/23/20,01/24/20,01/25/20,01/26/20,01/27/20,01/28/20,01/29/20,01/30/20,01/31/20,02/01/20,02/02/20,02/03/20,02/04/20,02/05/20,02/06/20,02/07/20,02/08/20,02/09/20,02/10/20,02/11/20,02/12/20,02/13/20,02/14/20,02/15/20,02/16/20,02/17/20,02/18/20,02/19/20,02/20/20,02/21/20,02/22/20,02/23/20,02/24/20,02/25/20,02/26/20,02/27/20,02/28/20,...,03/29/20,03/30/20,03/31/20,04/01/20,04/02/20,04/03/20,04/04/20,04/05/20,04/06/20,04/07/20,04/08/20,04/09/20,04/10/20,04/11/20,04/12/20,04/13/20,04/14/20,04/15/20,04/16/20,04/17/20,04/18/20,04/19/20,04/20/20,04/21/20,04/22/20,04/23/20,04/24/20,04/25/20,04/26/20,04/27/20,04/28/20,04/29/20,04/30/20,05/01/20,05/02/20,05/03/20,05/04/20,05/05/20,05/06/20,05/07/20
0,Afghanistan,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,120.0,170.0,174.0,237.0,273.0,281.0,299.0,349.0,367.0,423.0,444.0,484.0,521.0,555.0,607.0,665.0,714.0,784.0,840.0,906.0,933.0,996.0,1026.0,1092.0,1176.0,1279.0,1351.0,1463.0,1531.0,1703.0,1828.0,1939.0,2171.0,2335.0,2469.0,2704.0,2894.0,3224.0,3392.0,3563.0
1,Albania,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,212.0,223.0,243.0,259.0,277.0,304.0,333.0,361.0,377.0,383.0,400.0,409.0,416.0,433.0,446.0,467.0,475.0,494.0,518.0,539.0,548.0,562.0,584.0,609.0,634.0,663.0,678.0,712.0,726.0,736.0,750.0,766.0,773.0,782.0,789.0,795.0,803.0,820.0,832.0,842.0
2,Algeria,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,511.0,584.0,716.0,847.0,986.0,1171.0,1251.0,1320.0,1423.0,1468.0,1572.0,1666.0,1761.0,1825.0,1914.0,1983.0,2070.0,2160.0,2268.0,2418.0,2534.0,2629.0,2718.0,2811.0,2910.0,3007.0,3127.0,3256.0,3382.0,3517.0,3649.0,3848.0,4006.0,4154.0,4295.0,4474.0,4648.0,4838.0,4997.0,4997.0
3,Andorra,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,334.0,370.0,376.0,390.0,428.0,439.0,466.0,501.0,525.0,545.0,564.0,583.0,601.0,601.0,638.0,646.0,659.0,673.0,673.0,696.0,704.0,713.0,717.0,717.0,723.0,723.0,731.0,738.0,738.0,743.0,743.0,743.0,745.0,745.0,747.0,748.0,750.0,751.0,751.0,751.0
4,Angola,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,7.0,7.0,8.0,8.0,8.0,10.0,14.0,16.0,17.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,24.0,24.0,24.0,24.0,25.0,25.0,25.0,25.0,26.0,27.0,27.0,27.0,27.0,30.0,35.0,35.0,35.0,36.0,36.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,Japan,"Yamagata Prefecture, Japan",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,54.0,58.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,0.0
2093,Japan,"Yamanashi Prefecture, Japan",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,43.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,0.0,0.0
2094,Japan,"Shimane Prefecture, Japan",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [0]:
data = data[(data['Province/State'] != 'Diamond Princess') & (data['Province/State'] != 'Grand Princess')]
data_deaths = data_deaths[(data_deaths['Province/State'] != 'Diamond Princess') & (data_deaths['Province/State'] != 'Grand Princess')]

In [0]:
# Change format

#data = data.append(data_rus)
#data_deaths = data_deaths.append(data_deaths_rus)
data = data.groupby(["Country/Region","Province/State"]).sum()
data = data.stack().reset_index()
data.columns = ["Country/Region","Province/State","Date","Confirmed"]

data_deaths = data_deaths.groupby(["Country/Region","Province/State"]).sum()
data_deaths = data_deaths.stack().reset_index()
data_deaths.columns = ["Country/Region","Province/State","Date","Deaths"]

In [0]:
data_rus = data_rus.groupby(["Country/Region","Province/State"]).sum()
data_rus = data_rus.stack().reset_index()
data_rus.columns = ["Country/Region","Province/State","Date","Confirmed"]

data_deaths_rus = data_deaths_rus.groupby(["Country/Region","Province/State"]).sum()
data_deaths_rus = data_deaths_rus.stack().reset_index()
data_deaths_rus.columns = ["Country/Region","Province/State","Date","Deaths"]

In [0]:
data = data.merge(data_deaths, how='left', on=['Country/Region','Province/State','Date'])
data_rus = data_rus.merge(data_deaths_rus, how='left', on=['Country/Region','Province/State','Date'])

data.Date = pd.to_datetime(data.Date)
data.Confirmed = data.Confirmed.fillna(0).astype('int64')
data.Deaths = data.Deaths.fillna(0).astype('int64')

data_rus.Date = pd.to_datetime(data_rus.Date)
data_rus.Confirmed = data_rus.Confirmed.astype('int64')
data_rus.Deaths = data_rus.Deaths.astype('int64')

In [10]:
# You can get new data

main_timeseries = pd.read_csv('timeseries.csv')
main_timeseries = main_timeseries[['country','name','date','cases', 'deaths','level']]
main_timeseries = main_timeseries[main_timeseries['level'] == 'state']
del main_timeseries['level']
main_timeseries.columns = ['Country/Region','Province/State', 'Date', 'Confirmed', 'Deaths']
main_timeseries['Confirmed'] = main_timeseries['Confirmed'].fillna(0)
main_timeseries['Deaths'] = main_timeseries['Deaths'].fillna(0)
#data = data.append(main_timeseries)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# Countries and regions info

russia_regions = pd.read_csv("covid-19-solution/data/features/russia_regions.csv")
russia_regions['urban_pop_rate']=russia_regions['population_urban']/russia_regions['population']
russia_regions = russia_regions[['csse_province_state', 'population','population_urban','population_rural','urban_pop_rate', 'geoname_name']]
russia_regions.loc[russia_regions['geoname_name'] == 'Nenets', 'csse_province_state'] = 'Nenetskiy autonomous oblast'
russia_regions.loc[russia_regions['geoname_name'] == 'Chukotka', 'csse_province_state'] = 'Chukotskiy autonomous oblast'
russia_regions.loc[russia_regions['geoname_name'] == 'Altayskiy kray', 'csse_province_state'] = 'Altay republic'
del russia_regions['geoname_name']
russia_regions = russia_regions.rename({'csse_province_state': 'Province/State'})

qd=pd.read_csv("covid-19-solution/data/mobility/quarantine.csv")
qd=qd.append({'Start date': '2020-03-19','Country':'USA','Level':'National'}, ignore_index=True)
qd=qd.append({'Start date': '2020-03-16','Country':'CZE','Level':'National'}, ignore_index=True)
qd = qd[['Country', 'Start date']]
qd['Start date'] = pd.to_datetime(qd['Start date'])
qd=qd.groupby(['Country']).min().reset_index()
qd.columns = ["Country/Region", "Start_date"]

Health_systems = pd.read_csv("covid-19-solution/data/features/2.12_Health_systems.csv").groupby("Country_Region").mean()
Health_systems = Health_systems.fillna(Health_systems.mean())
Health_systems = Health_systems[["Health_exp_pct_GDP_2016","Health_exp_per_capita_USD_2016","Physicians_per_1000_2009-18","Nurse_midwife_per_1000_2009-18"]]
Health_systems = Health_systems.reset_index()
Health_systems.columns = ["Country/Region","Health_GDP","Health_USD","Physicians","Nurse"]

WorldPop = pd.read_csv("covid-19-solution/data/features/WorldPopulationByAge2020.csv")
WorldPop = WorldPop.fillna(WorldPop.mean())
WorldPop2 = (WorldPop[WorldPop["AgeGrp"]=="60+"]["PopTotal"].to_numpy() + WorldPop[WorldPop["AgeGrp"]=="40-59"]["PopTotal"].to_numpy())/(WorldPop[WorldPop["AgeGrp"]=="0-19"]["PopTotal"].to_numpy()+WorldPop[WorldPop["AgeGrp"]=="20-39"]["PopTotal"].to_numpy())
WorldPop = WorldPop.groupby("Location").mean()
WorldPop["Age_old/new"] = WorldPop2
WorldPop = WorldPop[["Age_old/new"]].reset_index()
WorldPop.columns = ['Country/Region', 'Age_old/new']

smoke = pd.read_csv("covid-19-solution/data/features/share-of-adults-who-smoke.csv")
smoke = smoke[smoke.Year > 2015].groupby("Entity").mean()
smoke = smoke[["Smoking prevalence, total (ages 15+) (% of adults)"]]
smoke.columns = ["Smoking"]
smoke = smoke.reset_index()
smoke.columns = ['Country/Region', 'Smoking']

countries2 = pd.read_csv("covid-19-solution/data/features/covid19countryinfo.csv")
countries2 = countries2.fillna(0).groupby("country").sum()
countries2 = countries2[["tests","testpop","gatheringlimit","hospibed","healthperpop"]]
countries2 = countries2.reset_index()
countries2.columns = ['Country/Region', 'tests', 'testpop', 'gatheringlimit', 'hospibed','healthperpop']

countries = pd.read_csv("covid-19-solution/data/features/countries.csv")
countries.drop(['iso_alpha2','iso_numeric','official_name','name'], axis='columns',inplace=True)
del countries['iso_alpha3']
countries.columns = ['Country/Region', 'density_n', 'fertility_rate', 'land_area',
       'median_age', 'migrants', 'population_coun', 'urban_pop_rate_c', 'world_share']

In [0]:
russia_regions = russia_regions.rename(columns={'csse_province_state':'Province/State'})
data2 = data.merge(Health_systems, how='left', on='Country/Region')
data2 = data2.merge(WorldPop, how='left', on='Country/Region')
data2 = data2.merge(smoke, how='left', on='Country/Region')
data2 = data2.merge(countries2, how='left', on='Country/Region')
data2 = data2.merge(countries, how='left', on='Country/Region')
data2 = data2.merge(qd, how='left', on='Country/Region')
data2 = data2.merge(russia_regions, how='left', on='Province/State')

In [0]:
data_rus2 = data_rus.merge(Health_systems, how='left', on='Country/Region')
data_rus2 = data_rus2.merge(WorldPop, how='left', on='Country/Region')
data_rus2 = data_rus2.merge(smoke, how='left', on='Country/Region')
data_rus2 = data_rus2.merge(countries2, how='left', on='Country/Region')
data_rus2 = data_rus2.merge(countries, how='left', on='Country/Region')
data_rus2 = data_rus2.merge(qd, how='left', on='Country/Region')
data_rus2 = data_rus2.merge(russia_regions, how='left', on='Province/State')

In [0]:
region_metadata = pd.read_csv('covid-19-solution/data/features/region_metadata.csv')
del region_metadata['density']
del region_metadata['population']
region_metadata = region_metadata.rename(columns={'Country_Region':'Country/Region', 'Province_State':'Province/State'})
region_metadata['Province/State'] = region_metadata['Province/State'].fillna('nan')
data2 = data2.merge(region_metadata, how='left', on=['Country/Region', 'Province/State'])

In [0]:
del region_metadata['Province/State']
data_rus2 = data_rus2.merge(region_metadata, how='left', on='Country/Region')

In [16]:
countries = pd.read_csv('covid-19-solution/data/features/countries.csv')
m_apple = pd.read_csv('covid-19-solution/data/mobility/mobility-apple.csv')
m_apple = m_apple[['date', 'apl-driving', 'apl-transit', 'apl-walking',
       'name']]
m_apple.columns = ['Date', 'apl-driving', 'apl-transit', 'apl-walking',
       'Country/Region']
m_apple['Date'] = pd.to_datetime(m_apple['Date'])
m_apple.sample()

,Date,apl-driving,apl-transit,apl-walking,Country/Region
6038,2020-04-20,39.45,NaN,31.13,Uruguay


In [0]:
countries = countries.rename(columns={'ccse_name':'country'})

In [18]:
m_google = pd.read_csv("covid-19-solution/data/mobility/mobility-google.csv")

'''countries = pd.read_csv("covid-19-solution/data/features/countries.csv")
countries.drop(['iso_alpha2','iso_numeric','official_name','name'], axis='columns',inplace=True)
del countries['iso_alpha3']
countries.columns = ['Country/Region', 'density_n', 'fertility_rate', 'land_area',
       'median_age', 'migrants', 'population_coun', 'urban_pop_rate_c', 'world_share']
       '''
m_google = m_google.merge(countries[['country','name']], how='left', on='country')
del m_google['country']
m_google.columns = ['Date','grocery-and-pharmacy', 'parks', 'residential', 'retail-and-recreation',
       'transit-stations', 'workplaces', 'Country/Region']
m_google['Date'] = pd.to_datetime(m_google['Date'])
m_google.sample()

,Date,grocery-and-pharmacy,parks,residential,retail-and-recreation,transit-stations,workplaces,Country/Region
2105,2020-03-20,-17.0,12.0,-16.0,-18.0,-5.0,2.0,NaN


In [0]:
russia_regions = pd.read_csv('covid-19-solution/data/features/russia_regions.csv')[['iso_code', 'name', 'type', 'name_with_type', 'csse_province_state',
       'federal_district', 'timezone', 'geoname_code', 'geoname_id',
       'geoname_name', 'population', 'population_urban', 'population_rural']]
russia_regions.columns = ['country', 'name', 'type', 'name_with_type', 'Province/State',
       'federal_district', 'timezone', 'geoname_code', 'geoname_id',
       'geoname_name', 'population_r', 'population_urban', 'population_rural']

In [20]:
m_yandex = pd.read_csv("covid-19-solution/data/mobility/mobility-yandex.csv")
m_yandex = m_yandex.merge(russia_regions, how='left', on=['country'])
del m_yandex['country']
m_yandex['Country/Region'] = 'Russia'
m_yandex.columns = ['Date', 'isolation', 'name', 'type', 'name_with_type', 'Province/State',
       'federal_district', 'timezone', 'geoname_code', 'geoname_id',
       'geoname_name', 'population', 'population_urban', 'population_rural',
       'Country/Region']
m_yandex['Date'] = pd.to_datetime(m_yandex['Date'])
m_yandex = m_yandex[['Date', 'isolation', 'Province/State',
       'federal_district', 'geoname_name',
       'Country/Region']]
m_yandex.sample()

,Date,isolation,Province/State,federal_district,geoname_name,Country/Region
4730,2020-04-21,2.6,Voronezh oblast,Центральный,Voronezj,Russia


In [0]:
data3 = data2.merge(m_apple, how='left', on=['Country/Region', 'Date'])
data3 = data3.merge(m_google, how='left', on=['Country/Region', 'Date'])
data3 = data3.merge(m_yandex, how='left', on=['Country/Region', 'Province/State', 'Date'])

data_rus3 = data_rus2.merge(m_apple, how='left', on=['Country/Region', 'Date'])
data_rus3 = data_rus3.merge(m_google, how='left', on=['Country/Region', 'Date'])
data_rus3 = data_rus3.merge(m_yandex, how='left', on=['Country/Region', 'Province/State', 'Date'])

In [0]:
data4 = data3.append(data_rus3)

In [0]:
data4['Health_GDP'] = data4['Health_GDP'].fillna(data4['Health_GDP'].mean())
data4['Health_USD'] = data4['Health_USD'].fillna(data4['Health_USD'].mean())
data4['Physicians'] = data4['Physicians'].fillna(data4['Physicians'].mean())
data4['Nurse'] = data4['Nurse'].fillna(data4['Nurse'].mean())
data4['Age_old/new'] = data4['Age_old/new'].fillna(data4['Age_old/new'].mean())

In [0]:
data4['Smoking'] = data4['Smoking'].fillna(data4['Smoking'].mean())
data4['tests'] = data4['tests'].fillna(data4['tests'].mean())
data4['testpop'] = data4['testpop'].fillna(data4['testpop'].mean())
data4['gatheringlimit'] = data4['gatheringlimit'].fillna(data4['gatheringlimit'].mean())
data4['healthperpop'] = data4['healthperpop'].fillna(data4['healthperpop'].mean())
data4['density_n'] = data4['density_n'].fillna(data4['density_n'].mean())
data4['fertility_rate'] = data4['fertility_rate'].fillna(data4['fertility_rate'].mean())

In [0]:
data4['land_area'] = data4['land_area'].fillna(data4['land_area'].mean())
data4['median_age'] = data4['median_age'].fillna(data4['median_age'].mean())
data4['migrants'] = data4['migrants'].fillna(data4['migrants'].mean())
data4['population_coun'] = data4['population_coun'].fillna(data4['population_coun'].mean())
data4['urban_pop_rate'] = data4['urban_pop_rate'].fillna(data4['urban_pop_rate'].mean())
data4['world_share'] = data4['world_share'].fillna(data4['world_share'].mean())
data4['hospibed'] = data4['hospibed'].fillna(data4['hospibed'].mean())

In [0]:
data4.loc[data4.hospibed == 0,"hospibed"] = data4.hospibed.mean()
data4.loc[data4.tests == 0,"tests"] = data4.tests.mean()
data4.loc[data4.testpop == 0,"testpop"] = data4.testpop.mean()
data4.loc[data4.gatheringlimit == 0,"gatheringlimit"] = data4.gatheringlimit.mean()
data4.loc[data4.hospibed == 0,"hospibed"] = data4.hospibed.mean()

In [0]:
data4['population'] = data4['population'].fillna(data4['population'].mean())
data4['population_urban'] = data4['population_urban'].fillna(data4['population_urban'].mean())
data4['population_rural'] = data4['population_rural'].fillna(data4['population_rural'].mean())
data4['urban_pop_rate'] = data4['urban_pop_rate'].fillna(data4['urban_pop_rate'].mean())

In [0]:
data4['apl-driving'] = data4['apl-driving'].fillna(data4['apl-driving'].mean())
data4['apl-transit'] = data4['apl-transit'].fillna(data4['apl-transit'].mean())
data4['apl-walking'] = data4['apl-walking'].fillna(data4['apl-walking'].mean())
data4['isolation'] = data4['isolation'].fillna(data4['isolation'].mean())
data4['grocery-and-pharmacy'] = data4['grocery-and-pharmacy'].fillna(data4['grocery-and-pharmacy'].mean())
data4['parks'] = data4['parks'].fillna(data4['parks'].mean())
data4['residential'] = data4['residential'].fillna(data4['residential'].mean())
data4['retail-and-recreation'] = data4['retail-and-recreation'].fillna(data4['retail-and-recreation'].mean())
data4['transit-stations'] = data4['transit-stations'].fillna(data4['transit-stations'].mean())
data4['workplaces'] = data4['workplaces'].fillna(data4['workplaces'].mean())
data4['isolation'] = data4['isolation'].fillna(data4['isolation'].mean())

In [0]:
data4['federal_district'] = data4['federal_district'].fillna('nan')
data4['geoname_name'] = data4['geoname_name'].fillna('nan')
data4['Start_date'] = data4['Start_date'].dt.dayofyear
data4['urban_pop_rate_c'] = data4['urban_pop_rate_c'].fillna(data4['urban_pop_rate_c'].mean())

In [0]:
russia_regions = russia_regions.rename(columns={'country':'Region'})

In [0]:
regions_ru_vec = pd.read_csv('covid-19-solution/data/features/regions_ru_vec.csv')

In [0]:
data4['Province/State'] = data4['Province/State'].fillna('nan')
regions_ru_vec['Region'] = regions_ru_vec['Region'].fillna('nan')
russia_regions['Province/State'] = russia_regions['Province/State'].fillna('nan')
russia_regions['Region'] = russia_regions['Region'].fillna('nan')

In [0]:
regions_ru_vec = regions_ru_vec.merge(russia_regions[['Region', 'Province/State']], how='left', on='Region')
del regions_ru_vec['Region']

In [0]:
regions_ru_vec_cols = regions_ru_vec.columns[:-1]

In [0]:
data4 = data4.merge(regions_ru_vec, how='left', on='Province/State')

In [0]:
for col in regions_ru_vec_cols:
  data4[col] = data4[col].fillna(data4[col].mean())

In [0]:
#data4.to_csv('covid-19-solution/data/data_with_features/_data_with_features.csv', index=False)

In [39]:
print(data4.shape)
data4.sample(5)

(232961, 101)


,Country/Region,Province/State,Date,Confirmed,Deaths,Health_GDP,Health_USD,Physicians,Nurse,Age_old/new,Smoking,tests,testpop,gatheringlimit,hospibed,healthperpop,density_n,fertility_rate,land_area,median_age,migrants,population_coun,urban_pop_rate_c,world_share,Start_date,population,population_urban,population_rural,urban_pop_rate,lat,lon,continent,area,apl-driving,apl-transit,apl-walking,grocery-and-pharmacy,parks,residential,retail-and-recreation,...,global_vec_16,global_vec_17,global_vec_18,global_vec_19,global_vec_20,global_vec_21,global_vec_22,global_vec_23,global_vec_24,global_vec_25,global_vec_26,global_vec_27,region_vec_0,region_vec_1,region_vec_2,region_vec_3,region_vec_4,region_vec_5,region_vec_6,region_vec_7,region_vec_8,region_vec_9,region_vec_10,region_vec_11,region_vec_12,region_vec_13,region_vec_14,region_vec_15,region_vec_16,region_vec_17,region_vec_18,region_vec_19,region_vec_20,region_vec_21,region_vec_22,region_vec_23,region_vec_24,region_vec_25,region_vec_26,region_vec_27
108098,Slovenia,"Ljutomer, Slovenia",2020-02-19,0,0,8.500000,1834.200000,3.000000,9.700000,1.317568,22.500000,6712.000000,309.734505,33.482838,4.600000,1315.094534,103.000000,1.600000,2.014000e+04,45.000000,2000.000000,2.078938e+06,0.550000,0.000300,NaN,1.753027e+06,1312777.8,440249.282353,0.709728,NaN,NaN,NaN,NaN,109.790000,72.823825,127.690000,NaN,NaN,NaN,NaN,...,0.417318,0.161792,0.273647,0.176004,0.156568,0.121104,0.104868,0.173756,0.391796,0.226819,0.487913,0.354939,0.307206,0.057515,0.087117,0.295813,0.156807,0.504937,0.149079,0.299047,0.072244,0.412258,0.132359,0.043938,0.020394,0.055131,0.112941,0.165954,0.43975,0.164325,0.204638,0.136935,0.14678,0.090775,0.06642,0.165818,0.418765,0.223213,0.499946,0.306906
223263,Vietnam,Vietnam,2020-03-23,123,0,5.700000,122.800000,0.800000,1.400000,1.032643,22.800000,9696.000000,10039.044860,20.000000,2.600000,3.431322,314.000000,2.100000,3.100700e+05,32.000000,-80000.000000,9.733858e+07,0.380000,0.012500,NaN,1.753027e+06,1312777.8,440249.282353,0.709728,NaN,NaN,NaN,NaN,65.780000,72.823825,65.010000,NaN,NaN,NaN,NaN,...,0.417318,0.161792,0.273647,0.176004,0.156568,0.121104,0.104868,0.173756,0.391796,0.226819,0.487913,0.354939,0.307206,0.057515,0.087117,0.295813,0.156807,0.504937,0.149079,0.299047,0.072244,0.412258,0.132359,0.043938,0.020394,0.055131,0.112941,0.165954,0.43975,0.164325,0.204638,0.136935,0.14678,0.090775,0.06642,0.165818,0.418765,0.223213,0.499946,0.306906
144764,United Kingdom,"City of Bristol, England, United Kingdom",2020-05-01,631,0,9.800000,3958.000000,2.100000,5.300000,1.023828,22.300000,50442.000000,1345.823143,33.482838,2.800000,61.058235,281.000000,1.800000,2.419300e+05,40.000000,260650.000000,6.788601e+07,0.830000,0.008700,83.0,1.753027e+06,1312777.8,440249.282353,0.709728,NaN,NaN,NaN,NaN,82.605326,72.823825,84.835839,NaN,NaN,NaN,NaN,...,0.417318,0.161792,0.273647,0.176004,0.156568,0.121104,0.104868,0.173756,0.391796,0.226819,0.487913,0.354939,0.307206,0.057515,0.087117,0.295813,0.156807,0.504937,0.149079,0.299047,0.072244,0.412258,0.132359,0.043938,0.020394,0.055131,0.112941,0.165954,0.43975,0.164325,0.204638,0.136935,0.14678,0.090775,0.06642,0.165818,0.418765,0.223213,0.499946,0.306906
174498,United States,"Denver County, Colorado, United States",2020-04-19,1784,0,8.145282,2443.245513,2.815329,7.279926,1.032643,21.800000,40218.117147,10403.318753,33.482838,5.479373,889.584838,172.530689,1.846099,3.190620e+06,39.649563,86384.955442,1.396698e+08,0.701473,0.017919,79.0,1.753027e+06,1312777.8,440249.282353,0.709728,NaN,NaN,NaN,NaN,82.605326,72.823825,84.835839,NaN,NaN,NaN,NaN,...,0.417318,0.161792,0.273647,0.176004,0.156568,0.121104,0.104868,0.173756,0.391796,0.226819,0.487913,0.354939,0.307206,0.057515,0.087117,0.295813,0.156807,0.504937,0.149079,0.299047,0.072244,0.412258,0.132359,0.043938,0.020394,0.055131,0.112941,0.165954,0.43975,0.164325,0.204638,0.136935,0.14678,0.090775,0.06642,0.165818,0.418765,0.223213,0.499946,0.306906
37745,Iraq,nan,2020-04-12,1352,0,3.300000,152

In [0]:
data4.to_csv('covid-19-solution/data/data_with_features/_data_with_features_all.csv', index=False)